<a href="https://colab.research.google.com/github/bozzobo/python/blob/main/XE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [257]:
# BASELINE IMPORTATION
import pandas as pd
import datetime


In [258]:
# BASELINE IMPORTATION MODULES
from datetime import timedelta, date

# BASELINE DEFINITION
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [327]:
# PERIOD for web data scraping
# Set period with a starting and an ending day

start_date = date(2024,10,15)
end_date = date(2024,10,16)

In [342]:
#DATAFRAME Iniialisation
#DF0
df = pd.DataFrame()
#DF1
df_cur0= pd.DataFrame()
#DF2
df_cur = pd.DataFrame()

In [343]:
 #DATAFRAME Definition
 #DF1
 df_cur = pd.read_html(f'https://www.xe.com/currencytables/?from=USD&date=2024-01-01')[0]
 df_cur = df_cur.drop(df_cur.columns[3], axis=1)
 df_cur = df_cur.drop(df_cur.columns[2], axis=1)
 df_cur = df_cur.drop(df_cur.columns[1], axis=1)
 #DF2
 cur0= {'Currency': ["USD", "EUR", "GBP","AED"]}
 df_cur0= pd.DataFrame(cur0)

In [344]:
#DF1 Currency defined Check
df_cur

,Currency
0,USD
1,EUR
2,GBP
3,INR
4,AUD
...,...
180,VED
181,XBT
182,XLM
183,XRP


In [276]:
#DF2 Currency defined Check
df_cur0

,Currency
0,USD
1,EUR
2,GBP
3,AED


In [345]:
#MAIN

for single_date in daterange(start_date, end_date):
#LOOP for each day in the period defined

  for i in range(len(df_cur0)):
  #LOOP for all the currency  defined in DF2
    cur = df_cur.iloc[i, 0]
    dfs = pd.DataFrame()
    dfs1 = pd.DataFrame()
    dfs2 = pd.DataFrame()

    sd = single_date.strftime("%Y-%m-%d")
    dfs = pd.read_html(f'https://www.xe.com/currencytables/?from={cur}&date={sd}')[0]

    for i in range(len(df_cur0)):
    #SUBLOOP for all the currency  defined in DF2
      curs = df_cur.iloc[i, 0]
      dfs1=dfs[dfs['Currency'].str.contains(curs)==True]#"(USD|EUR|GBP|AED|CAD|XAF|BRL|XAU|BTC|ETH")
      dfs2 = pd.concat([dfs2,dfs1], ignore_index=True)

    #DATA CLEAN 1
    #Column add
    dfs['Date'] = sd
    #Column rename
    dfs.rename(columns={dfs.columns[0]:'Unit Reiceived'}, inplace = True)
    dfs.rename(columns={dfs.columns[2]: "XE"}, inplace=True)
    #Column delete
    dfs = dfs.drop(dfs.columns[3], axis=1)
    dfs = dfs.drop(dfs.columns[1], axis=1)
    #Column insert
    dfs.insert(0,"Unit Deposed", cur)



    #LOOP to Exchange rate with the strongest currency first
    for i in range(len(dfs)):

      UnD = dfs.iloc[i, 0]
      UnR = dfs.iloc[i, 1]
      N = dfs.iloc[i, 2]

      if N < 1 :
        N = 1 / N
        dfs.iloc[i, 0] = UnR
        dfs.iloc[i, 1] = UnD
        dfs.iloc[i, 2] = N


    #DATA CLEAN 2

    #Create the rate symbol
    dfs["Rate"]=dfs["Unit Deposed"].astype(str)+"-"+ dfs["Unit Reiceived"]
    #Delete Duplicates
    dfs=dfs.drop_duplicates("Rate")

    #CONCATENATE all DFS
    df = pd.concat([df,dfs], ignore_index=True)

In [346]:
#DF0 Check Count
len(df)

740

In [347]:
#DF0 Check
df

,Unit Deposed,Unit Reiceived,XE,Date,Rate
0,USD,USD,1.000000e+00,2024-10-15,USD-USD
1,EUR,USD,1.089978e+00,2024-10-15,EUR-USD
2,GBP,USD,1.308282e+00,2024-10-15,GBP-USD
3,USD,INR,8.406105e+01,2024-10-15,USD-INR
4,USD,AUD,1.490673e+00,2024-10-15,USD-AUD
...,...,...,...,...,...
735,VED,INR,2.164372e+00,2024-10-15,VED-INR
736,XBT,INR,5.543670e+06,2024-10-15,XBT-INR
737,XLM,INR,7.779703e+00,2024-10-15,XLM-INR
738,XRP,INR,4.527900e+01,2024-10-15,XRP-INR


In [348]:
df.to_csv('XE.csv')

In [ ]:
#LOOP to delete value equal to 1
    #for i in range(len(dfs)):
      #if N == 1 :
        #dfs = dfs.drop(dfs.index[i], axis=0)

In [ ]:
#/content/exchange_rate_XE.csv